In [1]:
import os

import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt

# Load in our new data
This is notebook NO.2 in our world happiness series. Some notes on our new data:
1. Worlbank includes various attributes about countries over time
2. This data is formatted differently then our previous data
3. This data has much more NANs then our previous data
4. Our challenge will be fitting our new data to our old data's paradigm

In [2]:
# Our .csvs exist in our Worldbank_data folder
# Each csv holds data for a new feature, for now we will have a df for each

files = os.listdir("2 Worldbank_data")
new_features = [files[i].replace(".csv","") for i in range(len(files))]
new_features.append('Year')

additional_data = []
for file in files:
    additional_data.append(pd.read_csv("2 Worldbank_data\\" + file, header=2))

In [3]:
# Here are our feature names

new_features

['GNI_per_cap', 'Labor_force', 'Life_expectancy', 'Population', 'Year']

In [4]:
# This is what the data looks like

for data in additional_data:
    display(data.head(3))

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,22450.0,23520.0,24420.0,25190.0,26250.0,26560.0,26810.0,NaN,NaN,NaN
1,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,530.0,630.0,660.0,630.0,600.0,570.0,560.0,550.0,540.0,NaN
2,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3420.0,4170.0,4780.0,5010.0,4520.0,3770.0,3450.0,3210.0,3050.0,NaN


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,7499332.0,7891605.0,8307325.0,8741565.0,9176450.0,9566712.0,9947922.0,10334090.0,10699797.0,NaN
2,Angola,AGO,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,9910112.0,10270167.0,10646623.0,11034611.0,11437925.0,11848410.0,12268842.0,12705486.0,13164276.0,NaN


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,...,75.158,75.299,75.441,75.583,75.725,75.868,76.010,76.152,NaN,NaN
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.446,32.962,33.471,33.971,34.463,34.948,...,61.553,62.054,62.525,62.966,63.377,63.763,64.130,64.486,NaN,NaN
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.524,37.811,38.113,38.430,38.760,39.102,...,56.330,57.236,58.054,58.776,59.398,59.925,60.379,60.782,NaN,NaN


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN


In [5]:
# This is what we can expect our features to look like for each df

additional_data[0].columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', 'Unnamed: 64'],
      dtype='object')

In [6]:
# lets extract the features we care about
# aka we want to use the same years used in our last analysis
# We will then make 'Country' the index key and concatenate our data

desired_features = ['Country Name','2015', '2016', '2017', '2018']

for i in range(len(additional_data)):
    additional_data[i] = additional_data[i][desired_features]
    additional_data[i]["Feature_name"] = new_features[i]
    additional_data[i].rename(columns={"Country Name": "Country"}, inplace=True)
    additional_data[i].index = np.copy(additional_data[i]["Country"])
    additional_data[i].drop(["Country"], axis=1, inplace=True)

new_df = pd.concat(additional_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors

In [7]:
# Here is our current data

new_df.head(12)

,2015,2016,2017,2018,Feature_name
Aruba,26250.000000,26560.000000,26810.000000,NaN,GNI_per_cap
Afghanistan,600.000000,570.000000,560.000000,550.000000,GNI_per_cap
Angola,4520.000000,3770.000000,3450.000000,3210.000000,GNI_per_cap
Albania,4390.000000,4320.000000,4290.000000,4860.000000,GNI_per_cap
Andorra,NaN,NaN,NaN,NaN,GNI_per_cap
Arab World,7042.873351,6583.828058,6082.668627,6279.674858,GNI_per_cap
United Arab Emirates,42860.000000,40570.000000,39290.000000,41470.000000,GNI_per_cap
Argentina,12570.000000,12220.000000,13120.000000,12390.000000,GNI_per_cap
Armenia,4010.000000,3760.000000,3950.000000,4230.000000,GNI_per_cap
American Samoa,NaN,NaN,NaN,NaN,GNI_per_cap


In [31]:
"""Notice that this data is not the same shape as our previous data from 
our baseline_happiness_analysis. In this original notebook, we have one year
and multiple features per country, but in this dataset, we have multiple
years and one feature per country. Lets fix this."""

In [9]:
# indexes will hold the relative country name
# data_matrix will hold our newly shaped data

data_length = np.unique(new_df.index).shape[0]                  # length of unique countries
country_years = additional_data[0].columns.values.shape[0] - 1  # Num of years for each country excluding feature name
country_features = len(new_features)                            # Num of feature names in all new dfs

indexes = []
data_matrix = np.zeros((
    data_length, 
    country_years, 
    country_features))

In [10]:
# 1. First we will take subsets of our df by country
# 2. Then we will decompose this into raw data and its associated year and feature
# 3. Finally we will perform transformations on our data so that its orientation
#    matches the desired orientation of our data from the previous notebook

index_keys = new_df.index
for i in range(len(np.unique(new_df.index))):
    country_matrix = new_df.loc[index_keys[i]]
    data_years = country_matrix.columns.values[:-1]
    country_index = country_matrix.index.values[0]
    country_data = country_matrix.values[:, :len(additional_data)].T
    
    indexes.append(country_index)
    data_matrix[i, :, :-1] = country_data
    data_matrix[i, :, -1] = data_years

# We will package this new data in a master dataframe
# Each country is represented four times and oriented around our features uniformly

df_list = [pd.DataFrame(data_matrix[i], index=[indexes[i]] * data_matrix[i].shape[0]) for i in range(data_matrix.shape[0])]
new_df = pd.concat(df_list)
new_df.columns = new_features

new_df.head(12)

,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Aruba,26250.0,NaN,75.725,104341.0,2015.0
Aruba,26560.0,NaN,75.868,104872.0,2016.0
Aruba,26810.0,NaN,76.010,105366.0,2017.0
Aruba,NaN,NaN,76.152,105845.0,2018.0
Afghanistan,600.0,9176450.0,63.377,34413603.0,2015.0
Afghanistan,570.0,9566712.0,63.763,35383128.0,2016.0
Afghanistan,560.0,9947922.0,64.130,36296400.0,2017.0
Afghanistan,550.0,10334090.0,64.486,37172386.0,2018.0
Angola,4520.0,11437925.0,59.398,27884381.0,2015.0
Angola,3770.0,11848410.0,59.925,28842484.0,2016.0


# Lets (try to) merge our two datasets
It was apparent from the start of this project that I wanted to pull a bunch of data together to draw some type of conclusion. Using country data seemed like a great idea because of its widespread availability online. Although this is true, I was not ready for the data cleaning that would come from the mismatching and inconsistency of country names. I have outlined this learning experience below.

In [11]:
# Read in our old data

old_df = pd.read_csv('1 World_happiness_report_data/world_happiness_report.csv', index_col="Country")

In [12]:
def original_not_in_new(old_df, new_df):
    """find all items in our original data that do not exist in our new data
    """
    old_disjoint_new = [old_df.index[i] not in new_df.index for i in range(len(old_df))]

    return list(np.unique(sorted(old_df.index[old_disjoint_new])))

In [13]:
# Off the start, these countries exist in our original data but not in our new data

original_not_in_new(old_df, new_df)

['Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Egypt',
 'Gambia',
 'Hong Kong',
 'Iran',
 'Ivory Coast',
 'Kyrgyzstan',
 'Laos',
 'North Cyprus',
 'Palestinian Territories',
 'Russia',
 'Slovakia',
 'Somaliland Region',
 'South Korea',
 'Swaziland',
 'Syria',
 'Taiwan',
 'Venezuela',
 'Yemen']

In [14]:
# it turns out that many coutries in our new dataset are simply named differently
# this dictionary resprresents {new_df_name : original_df_name}

country_name_updates = {
    'Congo, Dem. Rep.' : 'Congo (Kinshasa)', 
    'Congo, Rep.' : 'Congo (Brazzaville)',
    'Egypt, Arab Rep.' : 'Egypt',
    'Gambia, The' : 'Gambia',
    'Hong Kong SAR, China' : 'Hong Kong',
    'Iran, Islamic Rep.' : 'Iran',
    "Cote d'Ivoire" : 'Ivory Coast',
    'Kyrgyz Republic' : 'Kyrgyzstan',
    'Lao PDR' : 'Laos',
    'West Bank and Gaza' : 'Palestinian Territories',
    'Russian Federation' : 'Russia',
    'Slovak Republic' : 'Slovakia',
    'Korea, Rep.' : 'South Korea',
    'Eswatini' : 'Swaziland',
    'Syrian Arab Republic' : 'Syria',
    'Venezuela, RB' : 'Venezuela',
    'Yemen, Rep.' : 'Yemen'
}

# Lets rename these countries to agree with our original data
new_df.rename(index=country_name_updates, inplace=True)

In [15]:
# after fixing all missmatched names, here are the countries remaining that need fixing

original_not_in_new(old_df, new_df)

['North Cyprus', 'Somaliland Region', 'Taiwan']

In [16]:
# we will simply drop these countres from our data

old_df.drop(['North Cyprus', 'Somaliland Region', 'Taiwan'], axis=0, inplace=True)

In [17]:
# Now we should have no missmatched names

original_not_in_new(old_df, new_df)

[]

In [18]:
# There are currently no countries in our original df that do not exist in our new df
# but what about the other way around? We need to remove all countries in our new
# df that are not in our original df

countries_to_remove = []
for country in np.unique(new_df.index):
    if country not in old_df.index:
        countries_to_remove.append(country)
        
display(countries_to_remove)

new_df.drop(countries_to_remove, axis=0, inplace=True)

['American Samoa',
 'Andorra',
 'Antigua and Barbuda',
 'Arab World',
 'Aruba',
 'Bahamas, The',
 'Barbados',
 'Bermuda',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Cabo Verde',
 'Caribbean small states',
 'Cayman Islands',
 'Central Europe and the Baltics',
 'Channel Islands',
 'Cuba',
 'Curacao',
 'Dominica',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD countries)',
 'East Asia & Pacific (excluding high income)',
 'Equatorial Guinea',
 'Eritrea',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Faroe Islands',
 'Fiji',
 'Fragile and conflict affected situations',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Grenada',
 'Guam',
 'Guinea-Bissau',
 'Guyana',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Isle of Man',
 'Kiribati',
 '

In [19]:
# Next we will need to remove NAN values
# Lets loop through our data and look at the data of countries that have NANs

for country in np.unique(new_df.index):
    country_df = new_df.loc[country]
    nan_location = np.where(np.sum(np.isnan(country_df.values)*1, axis=0))[0]
    if len(nan_location):
        display(new_df.loc[country])

,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Iran,5370.0,25359406.0,75.796,78492215.0,2015.0
Iran,5500.0,26559651.0,76.047,79564016.0,2016.0
Iran,5420.0,27419237.0,76.271,80673951.0,2017.0
Iran,NaN,27770616.0,76.479,81800269.0,2018.0


,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Kosovo,4010.0,NaN,71.346341,1788196.0,2015.0
Kosovo,3970.0,NaN,71.646341,1777557.0,2016.0
Kosovo,3970.0,NaN,71.946341,1791003.0,2017.0
Kosovo,4340.0,NaN,72.195122,1797085.0,2018.0


,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Somalia,NaN,3433878.0,55.920,13797201.0,2015.0
Somalia,NaN,3549907.0,56.324,14185613.0,2016.0
Somalia,NaN,3671052.0,56.709,14589119.0,2017.0
Somalia,NaN,3797583.0,57.068,15008154.0,2018.0


,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
South Sudan,1090.0,4476729.0,56.855,10715658.0,2015.0
South Sudan,NaN,4536023.0,57.120,10832512.0,2016.0
South Sudan,NaN,4580761.0,57.365,10910759.0,2017.0
South Sudan,NaN,4621451.0,57.604,10975920.0,2018.0


,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Syria,NaN,5285815.0,69.908,17997408.0,2015.0
Syria,NaN,5186809.0,70.315,17453933.0,2016.0
Syria,NaN,5133279.0,70.967,17068002.0,2017.0
Syria,NaN,5129679.0,71.779,16906283.0,2018.0


,GNI_per_cap,Labor_force,Life_expectancy,Population,Year
Venezuela,NaN,13520458.0,72.584,30081829.0,2015.0
Venezuela,NaN,13271796.0,72.405,29846179.0,2016.0
Venezuela,NaN,12945078.0,72.246,29390409.0,2017.0
Venezuela,NaN,12554309.0,72.128,28870195.0,2018.0


In [20]:
# Iran only has one NAN value we can fix this values by replacing 
# it with the average of values surrounding its position

display(new_df.loc['Iran']['GNI_per_cap'])

truth_val = new_df.loc['Iran', 'GNI_per_cap'].values[:-1]
new_df.loc['Iran', 'GNI_per_cap'] = np.append(truth_val, np.mean(truth_val))

display(new_df.loc['Iran']['GNI_per_cap'])

Iran    5370.0
Iran    5500.0
Iran    5420.0
Iran       NaN
Name: GNI_per_cap, dtype: float64

Iran    5370.0
Iran    5500.0
Iran    5420.0
Iran    5430.0
Name: GNI_per_cap, dtype: float64

In [21]:
# The remaining countries with NAN values cannot be fixed because
# they are missing an entire feature. We will remove these.

for country in np.unique(new_df.index):
    country_df = new_df.loc[country]
    nan_location = np.where(np.sum(np.isnan(country_df.values)*1, axis=0))[0]
    if len(nan_location):
        new_df.drop([country], axis=0, inplace=True)

In [22]:
# lets check if there are any more nulls

np.sum(new_df.isnull())

GNI_per_cap        0
Labor_force        0
Life_expectancy    0
Population         0
Year               0
dtype: int64

# Update original data
Before we concatenate our two datasets, it is important that each entry in dataset 'a' is accounted for in dataset 'b'. We will have to do some editing to our data to check this and drop the appropriate mismatched entries. 

In [23]:
def add_year_to_index(df):
    """Instead of searching through our data by country by year, we will append
    year to our country names. This allows each data entry to have its own
    unique key to be accessed with and appended to (when we concat our data).
    """
    df_year = df['Year'].values.astype('int')
    df_index = df.index
    df.index = [df_index[i] + "_" + str(df_year[i]) for i in range(len(df))]
    df.index.name = "Country_Year"

    df.drop(['Year'], axis=1, inplace=True)
    
    return df

# update indexes for both dfs

old_df = add_year_to_index(old_df)
new_df = add_year_to_index(new_df)

In [24]:
# lets take a look at the current state of our data

display(old_df.head())
display(new_df.head())

,Happiness,GDP,Family,Health,Freedom,Generosity,Trust_gov
Country_Year,,,,,,,
Switzerland_2015,1.000000,0.826132,0.962403,0.918244,0.993789,0.760595,0.372895
Iceland_2015,0.994524,0.770412,1.000000,0.924496,0.938841,0.256292,0.548198
Denmark_2015,0.987363,0.784113,0.970297,0.853099,0.969615,0.876175,0.428947
Norway_2015,0.986310,0.863099,0.949167,0.863409,1.000000,0.661394,0.435983
Canada_2015,0.966302,0.784592,0.943219,0.883326,0.945112,0.597144,0.575602


,GNI_per_cap,Labor_force,Life_expectancy,Population
Country_Year,,,,
Afghanistan_2015,600.0,9176450.0,63.377,34413603.0
Afghanistan_2016,570.0,9566712.0,63.763,35383128.0
Afghanistan_2017,560.0,9947922.0,64.130,36296400.0
Afghanistan_2018,550.0,10334090.0,64.486,37172386.0
Angola_2015,4520.0,11437925.0,59.398,27884381.0


In [25]:
# remove items in our old df that do not exist in our new df

for item in old_df.index:
    if item not in new_df.index:
        old_df.drop(item, axis=0, inplace=True)

In [26]:
# remove items in our new df that do not exist in our old df

for item in new_df.index:
    if item not in old_df.index:
        new_df.drop(item, axis=0, inplace=True)

In [27]:
# Our dfs should be of the same length

old_df.shape, new_df.shape

((598, 7), (598, 4))

In [28]:
# now we can save our clean, uniform datasets

old_df.to_csv('2 Cleaned_data/world_happiness.csv')
new_df.to_csv('2 Cleaned_data/worldbank.csv')

In [29]:
# Check that we can load in correctly

pd.read_csv('2 Cleaned_data/worldbank.csv', index_col="Country_Year").head()

,GNI_per_cap,Labor_force,Life_expectancy,Population
Country_Year,,,,
Afghanistan_2015,600.0,9176450.0,63.377,34413603.0
Afghanistan_2016,570.0,9566712.0,63.763,35383128.0
Afghanistan_2017,560.0,9947922.0,64.130,36296400.0
Afghanistan_2018,550.0,10334090.0,64.486,37172386.0
Angola_2015,4520.0,11437925.0,59.398,27884381.0


In [30]:
# Check that we can load in correctly

pd.read_csv('2 Cleaned_data/world_happiness.csv', index_col="Country_Year").head()

,Happiness,GDP,Family,Health,Freedom,Generosity,Trust_gov
Country_Year,,,,,,,
Switzerland_2015,1.000000,0.826132,0.962403,0.918244,0.993789,0.760595,0.372895
Iceland_2015,0.994524,0.770412,1.000000,0.924496,0.938841,0.256292,0.548198
Denmark_2015,0.987363,0.784113,0.970297,0.853099,0.969615,0.876175,0.428947
Norway_2015,0.986310,0.863099,0.949167,0.863409,1.000000,0.661394,0.435983
Canada_2015,0.966302,0.784592,0.943219,0.883326,0.945112,0.597144,0.575602
